In [2]:
#Dependencies
import timeit
from tqdm import tqdm
import pickle
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import missingno
import re

In [9]:
def instatiate_driver():
    #########################################################################################
    #Instatiate Selenium driver
    #Returns the handle object
    #########################################################################################
    chrome_options = webdriver.ChromeOptions()
    CHROMEDRIVER_PATH = executable_path
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
    return driver

In [18]:
url_longTerm_rental="https://www.kijiji.ca/b-apartments-condos/city-of-toronto"
url_longTerm_base="/c37l1700273"
pageNos = '/page-'
url_shortTerm_rental="https://www.kijiji.ca/b-short-term-rental/city-of-toronto"
url_shortTerm_base="/c42l1700273"
url_roomRoomates_rental="https://www.kijiji.ca/b-room-rental-roommate/city-of-toronto"
url_roomRoomates_base="/c36l1700273"
error_list=[]

In [19]:
def kijiji_getUrls(url, base):
    url_list=[]
    for i in tqdm(range(100)):
        url_final = url+pageNos+str(i)+base
        response = requests.get(url_final)
        soup = bs(response.text, "html.parser")
        results= soup.find_all("div", class_="search-item")
        for r in results:
            url_list.append("https://www.kijiji.ca"+r.a["href"])
    return(url_list)

In [20]:
url_list_longterm=kijiji_getUrls(url_longTerm_rental, url_longTerm_base)
url_list_longterm[0]

100%|██████████| 100/100 [05:07<00:00,  3.07s/it]


'https://www.kijiji.ca/v-apartments-condos/city-of-toronto/parkway-forest-towers-2-bedroom-1-bathroom-for-rent/1515948914'

In [21]:
#with open('kijiji_longtermRental.pkl', 'wb') as f:
 #   pickle.dump(url_list_longterm, f)

#with open('kijiji_longtermRental.pkl', 'rb') as f:
 #   kijiji_list_longterm = pickle.load(f)

In [23]:
#url_list_shortterm=kijiji_getUrls(url_shortTerm_rental, url_shortTerm_base)
#with open('kijiji_shorttermRental.pkl', 'wb') as f:
    #pickle.dump(url_list_shortterm, f)

with open('kijiji_shorttermRental.pkl', 'rb') as f:
    kijiji_list_shorttermRental = pickle.load(f)

#url_list_roomRoomates=kijiji_getUrls(url_roomRoomates_rental, url_roomRoomates_base)
#with open('kijiji_roomRoomatesRental.pkl', 'wb') as f:
 #   pickle.dump(url_list_roomRoomates, f)

with open('kijiji_roomRoomatesRental.pkl', 'rb') as f:
    kijiji_list_roomRoomatesRental = pickle.load(f)

In [24]:
url="https://www.kijiji.ca/v-apartments-condos/city-of-toronto/renovated-bachelor-for-rent-forest-hill/1508850924"
url2="https://www.kijiji.ca/v-apartments-condos/city-of-toronto/luxury-condo-2-bdrm-2-baths-yonge-st-clair/1516651347"
url3="https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/600-incredible-discount-200-less-per-month-from-august-15/1517738651"
url4= "https://www.kijiji.ca/v-short-term-rental/city-of-toronto/downtown-fully-furnished-yonge-front/1517081941"

In [44]:
def kijiji_url_scrape_LongTerm(kijiji_url):
    try:
        #driver = webdriver.Chrome()
        #driver.get(kijiji_url)
        response = requests.get(kijiji_url)
        soup = bs(response.text, "html.parser")
        time.sleep(1)
        #page_source = driver.page_source
        #soup=bs(page_source, "html.parser")

        ad_id= soup.find("a", class_="adId-4111206830").text.strip()
        
        images=[]
        img_result=soup.find_all("img", class_="heroImageForPrint-1152389693")
        for img in img_result:
            images.append(img["src"])
        title= soup.find("h1", class_="title-2323565163").text.strip()
        address= soup.find("span", class_="address-3617944557").text.strip()
        price= soup.find("div", class_="priceWrapper-1165431705").span.text.strip()
        post_published_date= soup.find("div", class_="datePosted-383942873").time["datetime"]
        rental_type= soup.find_all("span", class_="noLabelValue-3861810455")[0].text.strip()
        bedroom_no= soup.find_all("span", class_="noLabelValue-3861810455")[1].text.strip()
        bathroom_no= soup.find_all("span", class_="noLabelValue-3861810455")[2].text.strip()
        list_results= soup.find_all("ul", class_="list-1757374920")[1]

        utilities_list=[]
        utilities= list_results.find_all("svg")
        for util in utilities:
            utilities_list.append(util["aria-label"])
        if(len(utilities_list)==0):
            utilities_list.append("Not Included")
        wifi_and_more=soup.find_all("ul", class_="list-1757374920")[2].text.strip()


        appliances=[]
        results= soup.find_all("ul", class_="list-1757374920")[4]
        for r in results:
            if(r.text):
                appliances.append(r.text.strip())

        personal_outdoor_space= soup.find_all("ul", class_="list-1757374920")[5].text.strip()

        amenities= []
        results= soup.find_all("ul", class_="list-1757374920")[7]
        for r in results:
            if(r.text):
                amenities.append(r.text.strip())
        description= soup.find("div", class_="descriptionContainer-3544745383").p.text.strip()

        dict_attributes={
            'ad_id': ad_id,
            'images': images,
            'title': title,
            'address':address,
            'price': price,
            'post_published_date': post_published_date,
            'rental_type': rental_type,
            'bedroom_no': bedroom_no,
            'bathroom_no': bathroom_no,
            'utilities_list': utilities_list,
            'wifi_and_more': wifi_and_more,
            'appliances':appliances,
            'personal_outdoor_space': personal_outdoor_space,
            'amenities': amenities,
            'description': description,
            'url': kijiji_url
        }

        dt_list=soup.find_all("dt")
        for i in range(0,len(dt_list)):
            dict_attributes[soup.find_all("dt")[i].text]=soup.find_all("dd")[i].text
    except:
        #error_list.append(kijiji_url)
        return(kijiji_url)
    return(dict_attributes)


In [45]:
#kijiji_list_scrape_LongTerm(url2)

In [46]:
def kijiji_url_scrape_other(kijiji_url):
    try:
        #driver = webdriver.Chrome()
        #driver.get(kijiji_url)
        response = requests.get(kijiji_url)
        soup = bs(response.text, "html.parser")
        time.sleep(1)
        #page_source = driver.page_source
        #soup=bs(page_source, "html.parser")

        ad_id= soup.find("a", class_="adId-4111206830").text.strip()
        
        images=[]
        img_result=soup.find_all("img", class_="heroImageForPrint-1152389693")
        for img in img_result:
            images.append(img["src"])
            
        title= soup.find("h1", class_="title-2323565163").text.strip()
        address= soup.find("span", class_="address-3617944557").text.strip()
        price= soup.find("span", class_="currentPrice-2842943473").span.text.strip()
        post_published_date= soup.find("div", class_="datePosted-383942873").time["datetime"]
        rental_type= soup.find("h1", class_="crumbH1-3559413592").span.text.strip()
        description= soup.find("div", class_="descriptionContainer-3544745383").p.text.strip()
        dict_attributes={
            'ad_id': ad_id,
            'images': images,
            'title': title,
            'address':address,
            'price': price,
            'post_published_date': post_published_date,
            'rental_type': rental_type,
            'description': description,
            'url': kijiji_url
        }

        dt_list=soup.find_all("dt")
        for i in range(0,len(dt_list)):
            dict_attributes[soup.find_all("dt")[i].text]=soup.find_all("dd")[i].text
    except:
        #error_list.append(kijiji_url)
        return(kijiji_url)
    return(dict_attributes)


In [47]:
def kijiji_scrape():
    records=[]
    for url in tqdm(kijiji_list_longterm):
        records.append(kijiji_url_scrape_LongTerm(url))
        
    for url in tqdm(kijiji_list_shorttermRental):
        records.append(kijiji_url_scrape_other(url))
        
    for url in tqdm(kijiji_list_roomRoomatesRental):
        records.append(kijiji_url_scrape_other(url))
    return(records)

In [ ]:
records= kijiji_scrape()

 16%|█▋        | 734/4500 [37:19<16:28:19, 15.75s/it]

In [55]:
#with open('kijiji_housing_enriched.pkl', 'wb') as f:
#    pickle.dump(records, f)

with open('kijiji_housing_enriched.pkl', 'rb') as f:
    kijiji_renting_data = pickle.load(f)

In [63]:
kijiji_clean_data=[]
for listing in kijiji_renting_data:
    if str(type(listing))!="<class 'str'>":
        print(type(listing))
        kijiji_clean_data.append(listing)

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

In [64]:
DF = pd.DataFrame(kijiji_clean_data)
DF.head()

,ad_id,images,title,address,price,post_published_date,rental_type,bedroom_no,bathroom_no,utilities_list,...,Agreement Type,Move-In Date,Pet Friendly,Size (sqft),Furnished,Air Conditioning,Smoking Permitted,Barrier-free Entrances and Ramps,Visual Aids,Accessible Washrooms in Suite
0,1515948914,[https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...,"Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...","100 Parkway Forest Drive, Toronto, ON, M2J 1L6","$2,209.00",2020-08-13T22:34:37.000Z,Apartment,Bedrooms: 2,Bathrooms: 1,"[No: Hydro, Yes: Heat, Yes: Water]",...,1 Year,"August 13, 2020",Yes,940,No,No,Yes,NaN,NaN,NaN
1,1505801433,[https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/jms...,Open House August 12th from 7:00pm to 9:00pm w...,"70 Spadina Road, Toronto, ON, M5R 2T6","$1,587.00",2020-08-13T14:02:28.000Z,Apartment,Bedrooms: Bachelor/Studio,Bathrooms: 1,"[No: Hydro, Yes: Heat, Yes: Water]",...,1 Year,"September 1, 2020",Yes,Not Available,No,No,Outdoors only,NaN,NaN,NaN
2,1502301137,[https://i.ebayimg.com/00/s/NjAwWDgwMA==/z/cIk...,"One, two bedroom large unit from $1650 , Hydro...","40 Tuxedo Ct, Scarborough, ON M1G 3S7, Canada","$1,650.00",2020-07-17T14:08:19.000Z,Apartment,Bedrooms: 1,Bathrooms: 1,"[No: Hydro, Yes: Heat, Yes: Water]",...,1 Year,"July 24, 2020",Limited,750,No,No,Outdoors only,NaN,NaN,NaN
3,1505788420,[https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/NmU...,1 Bedroom Renovated Apartment for Rent - 135 T...,"135 Tyndall Avenue, Toronto, ON, M6K 2G4","$1,949.00",2020-08-13T19:06:36.000Z,Apartment,Bedrooms: 1,Bathrooms: 1,"[No: Hydro, Yes: Heat, Yes: Water]",...,1 Year,"August 13, 2020",Yes,626,No,No,No,NaN,NaN,NaN
4,1516085720,[https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/fcY...,"Yonge and Finch, 1+1 unit , spacious and brigh...","M2N 5R6, Canada","$2,100.00",2020-08-06T02:49:14.000Z,Condo,Bedrooms: 1 + Den,Bathrooms: 1,"[No: Hydro, Yes: Heat, Yes: Water]",...,1 Year,"August 4, 2020",No,675,No,Yes,No,Yes,No,Yes


In [65]:
DF.shape

(737, 27)